# Usando la API de DeepSeek - Reasoner
https://api-docs.deepseek.com/guides/reasoning_model


 🔍 **¿Qué es DeepSeek Reasoner?**  
DeepSeek es una plataforma de IA avanzada diseñada para ofrecer soluciones inteligentes y personalizadas en tiempo real.  
Con modelos de lenguaje de última generación, DeepSeek te permite:  

DeepSeek Reasoning Model es un modelo avanzado de razonamiento y generación de lenguaje desarrollado por DeepSeek, 
diseñado para realizar tareas complejas que requieren comprensión profunda, lógica y capacidad de inferencia.  
A diferencia de los modelos de lenguaje tradicionales, está optimizado para:

✅ Razonamiento paso a paso  
✅ Alta precisión en tareas complejas  
✅ Contexto extendido (hasta 128K tokens)  
✅ Optimización para respuestas estructuradas    

In [1]:
# Importa el cliente estandar de OpenAI, se conecta DIRECTAMENTE a los servicios de OpenAI como : GPT-3, GPT-4, DALL-E, etc.
from openai import OpenAI
# Librerias que permite interactuar con el Sistema Operativo. Administrar archivos, variables de entorno y más.
import os

In [2]:
os.environ["DEEPSEEK_API_KEY"] = "sk-ebed09dc4b2242bebd1e21cd3be686d3"  # ¡Mantén tus claves seguras!

### Deepseek-Chat
Diseñado para respuestas rápidas y directas, ideal para interacciones conversacionales estándar.

In [3]:
client = OpenAI(
    api_key=os.environ["DEEPSEEK_API_KEY"],  # Clave API de OpenAI.
    base_url="https://api.deepseek.com"  # URL base de la API. Por defecto https://api.openai.com/v1, puedes cambiarla si estás utilizando otra solucion.
    )

# El metodo client.chat.completions.create es una funcion proporcionada por la biblioteca OpenAI
# Facilita la interaccion con modelos de lenguaje basados en chat, como GPT-4 u otros. 
# Permite enviar una solicitud a la API de OpenAI (o a una API compatible, como DeepSeek) para generar respuestas de chat en formato de conversacion.
response = client.chat.completions.create(
    model="deepseek-chat",  # Nombre del modelo a utilizar como: "gpt-4-turbo", "deepseek-chat"
    # Lista de mensajes del chat en formato [{role: "user", content: "Hola"}]
    messages=[  
        {"role": "system", "content": "Eres un profesor de matematicas"},
        {"role": "user", "content": "9.11 o 9.8, cual es mayor?"}
    ],    
    stream=False,  # (True) la respuesta del modelo se envía en fragmentos (streaming) y (False) la respuesta del modelo como un bloque completo.
    #max_tokens= 50,  # Límite máximo de tokens en la respuesta generada.
    temperature = 0.7,  # Controla la aleatoriedad. Rango (0 a 2). Respuestas más deterministas (0.0 - 0.3) , respuestas muy aleatorias (1.3 - 2.0).
    timeout=30 # Cancela la conexión si transcurre 30 segundos sin recibir la respuesta completa.
)

resultado = response.choices[0].message
print("Respuesta:", resultado.content)
print("--------------------------------------------")
print("Tokens usados:", response.usage.total_tokens)

Respuesta: Para determinar cuál es mayor entre **9.11** y **9.8**, seguimos estos pasos:

1. **Igualar el número de decimales**:
   - Podemos reescribir **9.8** como **9.80** para facilitar la comparación.

2. **Comparar las partes enteras**:
   - Ambas cifras tienen **9** como parte entera, por lo que pasamos a comparar los decimales.

3. **Comparar las partes decimales**:
   - **9.11** tiene **11** centésimas.
   - **9.80** tiene **80** centésimas.
   
   Dado que **80 > 11**, entonces **9.80 > 9.11**.

**Respuesta final**:
\[
\boxed{9.8 \text{ es mayor que } 9.11}
\]
--------------------------------------------
Tokens usados: 216


### Deepseek-Reasoner
Optimizado para tareas que requieren un análisis más profundo y detallado, lo que resulta en tiempos de respuesta más largos.

In [4]:
client = OpenAI(
    api_key=os.environ["DEEPSEEK_API_KEY"], 
    base_url="https://api.deepseek.com"
    )

messages = [{"role": "system", "content": "Eres un profesor"},
            {"role": "user", "content": "9.11 o 9.8, cual es mayor?"}
           ]
response = client.chat.completions.create(
    model="deepseek-reasoner",
    messages= [
        {"role": "system", "content": "Eres un profesor"},
        {"role": "user", "content": "9.11 o 9.8, cual es mayor?"}
],
)

reasoning_content = response.choices[0].message.reasoning_content
content = response.choices[0].message.content

In [5]:
print("Reasoning:", reasoning_content)

Reasoning: Tengo una pregunta del usuario: "9.11 o 9.8, ¿cuál es mayor?" Parece una comparación simple entre dos números decimales. El usuario podría estar confundido porque 9.11 tiene más dígitos después del punto decimal, pero eso no necesariamente lo hace mayor.

Primero, recordemos cómo comparar números decimales. Comparamos la parte entera primero. Ambos números tienen 9 como parte entera, así que eso no ayuda. Luego, comparamos el primer dígito después del decimal. En 9.11, el primer dígito decimal es 1, y en 9.8, es 8. Como 8 es mayor que 1, eso significa que 9.8 es mayor que 9.11.

Puedo pensarlo en términos de fracciones. 9.11 es 911/100, y 9.8 es 98/10, que es lo mismo que 980/100. Comparando 911/100 y 980/100, claramente 980/100 es mayor.

Otra forma es alinear los decimales. 9.8 es lo mismo que 9.80. Entonces comparamos 9.80 y 9.11. 80 centésimas es mayor que 11 centésimas, así que 9.80 > 9.11.

El usuario podría estar pensando en fechas o algo así, como el 11 de septiembre

In [6]:
print("Respuesta:", content)

Respuesta: **9.8 es mayor que 9.11**.  

### Explicación sencilla:  
- **9.8** equivale a **9.80** (si añadimos un cero para comparar).  
- **9.80** tiene **80 centésimas**, mientras que **9.11** tiene **11 centésimas**.  
- Como **80 > 11**, entonces **9.8 > 9.11**.  

### ¿Por qué a veces genera confusión?  
Algunas personas piensan que "más dígitos después del punto decimal significa un número mayor", pero esto **no es cierto**. En los decimales, el valor depende de la posición:  
- **9.8** = 9 + 8/10 = **9.80** (980 centésimas).  
- **9.11** = 9 + 1/10 + 1/100 = **911 centésimas**.  
Como **980 > 911**, queda claro que **9.8 es mayor**.  

¡Espero que te ayude! 😊


#### Streaming

In [7]:
client = OpenAI(
    api_key=os.environ["DEEPSEEK_API_KEY"], 
    base_url="https://api.deepseek.com"
    )

messages = [{"role": "system", "content": "Eres un profesor"},
            {"role": "user", "content": "9.11 o 9.8, cual es mayor?"}
           ]
response = client.chat.completions.create(
    model="deepseek-reasoner",
    messages=messages,
    stream=True
)

print("========== RAZONAMIENTO EN STREAMING =============")
respuesta_actual = ""  # Acumulador para la respuesta

for chunk in response:
    if not chunk.choices:
        continue
    
    delta = chunk.choices[0].delta
    
    if getattr(delta, 'reasoning_content', None):
        print(delta.reasoning_content, end='', flush=True)
    
    if getattr(delta, 'content', None):
        respuesta_actual += delta.content  # Acumula sin imprimir

# Imprime la respuesta completa al final
if respuesta_actual:
    print(f"\n\n========== RESPUESTA ==========\n{respuesta_actual}")

========== RAZONAMIENTO EN STREAMING =============
Tengo una pregunta del usuario: "9.11 o 9.8, ¿cuál es mayor?" Parece que el usuario está comparando dos números decimales, 9.11 y 9.8. Necesito determinar cuál es mayor.

Primero, recordemos cómo comparar decimales. Los decimales se comparan dígito a dígito de izquierda a derecha. Ambos números tienen un 9 en la parte entera, así que eso no ayuda. Ahora, mirando el primer dígito después del punto decimal.

Para 9.11, el primer dígito decimal es 1. Para 9.8, el primer dígito decimal es 8. Como 8 es mayor que 1, eso sugeriría que 9.8 es mayor que 9.11. Pero quiero asegurarme.

Puedo pensar en los números como fracciones o alinearlos decimalmente. Vamos a alinearlos:

9.11

9.80  (porque 9.8 es lo mismo que 9.80)

Ahora, comparando:

- Unidades: 9 y 9, iguales.

- Décimas: 1 (de 9.11) y 8 (de 9.80). 8 es mayor que 1, así que 9.80 es mayor.

Para confirmar, puedo convertirlos a centésimas:

9.11 = 911/100

9.8 = 98/10 = 980/100 (multiplica